# Horse Racing Project

### Part 1: Create Dataset

This is Part 1 of a project where I create the dataset containing selected features of horse racing at Warsaw's Sluzewiec Track, which dates back to 1995 and contains all of the results history up to 2018. The data is collected from https://koniewyscigowe.pl. 

In Part 2 I will attempt to perform Exploratory Data Analysis on this dataset.

In [1]:
## Importing necessary modules
import numpy as np
import pandas as pd
import re # regular expressions
import csv # working with CSV
from bs4 import BeautifulSoup, SoupStrainer # for html objects
import time # time the code
import string
import httplib2 # html parser
pd.set_option('display.max_columns', 500) ## to see all the columns
pd.set_option('display.max_rows', 500) ## To see all the rows
import string

## Horses data frame

The horses are listed on a web page in alphabetical order. We need to loop through every letter to scrap all the horses IDs and names. The links are the only thing that is interesting at this step, as they contain the information that will be used in further process.

In [2]:
%%time
horses = [] # create empty list
it = 0
size = len(list(string.ascii_uppercase))-1
for letter in list(string.ascii_uppercase):
    
    # Loop through every letter and find the links
    http = httplib2.Http()
    status, response = http.request('https://koniewyscigowe.pl/spis_koni_wyscigowych?l='+letter)
    for link in BeautifulSoup(response, 'html.parser', parse_only=SoupStrainer('a')):
        if link.has_attr('href'):
            if (re.findall(r'\b'+'horse'+r'\b',link['href'])):
                horses.append(link['href'])
                          
    # Progress bar
    string = '['
    for k in range(int(it / size * 100)):
        string += '#'
    for k in range(100 - int(it / size * 100)):
        string += '-'
    string += ']'
    string += '{0:.2f}'.format(it / size * 100) + "%"
    print('\r' + string, sep='', end=' ', flush=True)
    it += 1
print('\n')

[####################################################################################################]100.00% 

Wall time: 11.8 s


The progress bar shows the percentage of how many more letters are left to process. The duration of the running code is timed and presented at the end. 

We can check how many link were found in the process and show for reference the first 10 of them:

In [3]:
len(horses)

17396

In [4]:
horses[0:10]

['/horse/20899-a-dee-joe',
 '/horse/12327-a-lot-of-mary',
 '/horse/20588-a-nihala',
 '/horse/23514-aaren-al-khalediah',
 '/horse/21448-alatune',
 '/horse/4655-af-pompey',
 '/horse/13166-aaran',
 '/horse/354-ababy',
 '/horse/12892-abadan',
 '/horse/4480-abakan']

### Horses names
The names of the horses are assigned by splitting them from the previous step:

In [5]:
%%time
names = [] # create an empty list
for i in range(0,len(horses)):
    name = ''
    temp = horses[i].split('-')
    horses[i] = temp[0]
    if len(temp)>1:
        for j in range(1,len(temp)):
            name += temp[j]+' '
    names.append(name[0:-1].title())

Wall time: 28 ms


In [6]:
len(names)

17396

In [7]:
names[0:10]

['A Dee Joe',
 'A Lot Of Mary',
 'A Nihala',
 'Aaren Al Khalediah',
 'Alatune',
 'Af Pompey',
 'Aaran',
 'Ababy',
 'Abadan',
 'Abakan']

The length is the same as the length of the horses list as expected, and the first ten names are shown for reference.

Based on the links we previously found we can perform the web scraping process to acquire more information about the horses. We check what kind of information we have on the webpage https://koniewyscigowe.pl.

For exploratory purposes we use the horse with the ID number of 23514 and named Aaren al Khalediah. The al Khalediah horses are well known at Sluzewiec, there are many of them and with great results. 

In [8]:
r = pd.read_html('https://koniewyscigowe.pl'+'/horse/23514-aaren-al-khalediah')

In [9]:
r[0][0]

0     Kasztanowaty ogier ur. 28.04.2015 (4 lata)
1                                          Rasa:
2                                   Pochodzenie:
3                                        Ojciec:
4                                         Matka:
5                                        Trener:
6                                    Właściciel:
7                                       Hodowca:
8                                       Wymiary:
9                                       Stajnia:
10                                          suma
Name: 0, dtype: object

In [10]:
# The information about: color, gender, date of birth, age
r[0][0][0]

'Kasztanowaty ogier ur. 28.04.2015 (4 lata)'

In [11]:
# The information about: breed
r[0][1][1]

'Czysta krew arabska'

In [12]:
# The information about: country of origin
r[0][1][2]

'Francja'

In [13]:
# The information about: father
r[0][1][3]

'Nayef Al Khalidiah'

In [14]:
# The information about: mother
r[0][1][4]

'Shaza Al Khalediah / Khalid El Biwaibiya'

In [15]:
# The information about: trainer
r[0][1][5]

'M. Kacprzyk'

In [16]:
# The information about: owner
r[0][1][6]

'Polska AKF Sp. z o.o.'

In [17]:
# The information about: breeder
r[0][1][7]

'Al Khalediah Stables'

In [18]:
# The information about: size (the dimensions are explained later)
r[0][1][8]

'154.0-179.0-18.50'

In [19]:
# The information about: stables
r[0][1][9]

'Polska AKF Sp. z o.o.'

Based on this information we can think of how we want our data frame to look like. The information about the color and gender of the horse, its age, breed, country of origin, parents, the person who is responsible for the training, owner, breeder, stables and size can be the features which describe the horse. With that information we can build the data frame.

### Create the Horses Data Frame
With the available information we build the first data frame for horses.

In [20]:
horses_DF = pd.DataFrame(columns = ['id_name', 'gender_age', 'breed', 'country',
                                   'father', 'mother', 'trainer', 'owner', 
                                   'breeder', 'size', 'stables'])
horses_DF

,id_name,gender_age,breed,country,father,mother,trainer,owner,breeder,size,stables


We then fill the data frame:

In [ ]:
%%time
#it = 0
size = len(horses)
r = []
for i in range(0,size):
    r = pd.read_html('https://koniewyscigowe.pl'+horses[i])
    horses_DF.at[i,'id_name'] = horses[i]
    horses_DF.at[i,'gender_age'] = r[0][0][0]
    for j in range(1,10):
        if r[0][0][j] == 'Rasa:':
            horses_DF.at[i,'breed'] = r[0][1][j]
        if r[0][0][j] == 'Pochodzenie:':
            horses_DF.at[i,'country'] = r[0][1][j]
        if r[0][0][j] == 'Ojciec:':
            horses_DF.at[i,'father'] = r[0][1][j]
        if r[0][0][j] == 'Matka:':
            horses_DF.at[i,'mother'] = r[0][1][j]
        if r[0][0][j] == 'Trener:':
            horses_DF.at[i,'trainer'] = r[0][1][j]
        if r[0][0][j] == 'Właściciel:':
            horses_DF.at[i,'owner'] = r[0][1][j]
        if r[0][0][j] == 'Hodowca:':
            horses_DF.at[i,'breeder'] = r[0][1][j]
        if r[0][0][j] == 'Wymiary:':
            horses_DF.at[i,'size'] = r[0][1][j]
        if r[0][0][j] == 'Stajnia:':
            horses_DF.at[i,'stables'] = r[0][1][j]
    
    # Progress bar
    string = '['
    for k in range(int(it / size * 100)):
        string += '#'
    for k in range(100 - int(it / size * 100)):
        string += '-'
    string += ']'
    string += '{0:.2f}'.format(it / size * 100) + "%"
    print('\r' + string, sep='', end=' ', flush=True)
    it += 1
print('\n')
print('Time: ')

For some reason the webpage for horse with id 5011 is generating an internal error.
Try: https://koniewyscigowe.pl/horse/5011.
This record will be removed from the data frame.

The id_name column needs to be sorted out and divided into id and name, the gender_age feature needs to be divided into separate columns. Some data is in Polish like: breed, country, trainer, gender, color, and needs to be translated.

Let's get to work:

Delete the unnecessary part of the id_name data. The length of the '/horse/' is always 7 characters long and we remove this part:

In [ ]:
for i in range(0,len(horses_DF)):
    horses_DF['id_name'].iloc[i] = horses_DF['id_name'].iloc[i][7:]

In [ ]:
horses_DF.head(2)

### Names:

We create new column with the horses names information based on the data we collected before:

In [ ]:
horses_DF['name'] = names

In [ ]:
horses_DF.head(2)

And export the results:

In [ ]:
horses_DF.to_csv(r'.\horses_data\horses_DF.csv', encoding="utf-8")

### Breeds and countries:

Let's check how many different breeds and countries exist in the data frame:

In [23]:
len(horses_DF.groupby('breed').sum())

9

In [24]:
len(horses_DF.groupby('country').sum())

35

### Date of birth - horse age

The horse age comes from the gender_age data. We are interested only in the year of the birth, that is way we search for a 4 digit sequence in the gender_age data and append to the age list.

In [ ]:
%%time
age = []
for i in range(0,len(horses_DF)):
    ke_ = re.findall(r'\d{4}',str(horses_DF['gender_age'].iloc[i]))
    age.append(ke_)

Assign the age list to the date of birth `dob` column:

In [ ]:
horses_DF['dob'] = age

Check the results and export to csv file:

In [ ]:
horses_DF.head(2)

In [ ]:
horses_DF.to_csv(r'.\horses_data\horses_DF.csv', encoding="utf-8")

Some more cleaning is needed:

In [ ]:
%%time
for i in range(0,len(horses_DF)):
    if horses_DF['dob'][i]:
        horses_DF['dob'][i] = horses_DF['dob'][i][2:6]

And the count of the horses based on date of birth is shown:

In [26]:
horses_DF.groupby('dob').count()

,Unnamed: 0,id_name,breed,country,father,mother,trainer,owner,breeder,size,stables,name,gender,color
dob,,,,,,,,,,,,,,
1986.0,2,2,2,2,2,2,2,2,2,1,2,2,2,2
1987.0,1,1,1,1,1,1,1,1,1,0,1,1,1,1
1988.0,7,7,7,7,7,7,7,7,7,7,7,7,7,7
1989.0,12,12,12,12,12,12,12,12,12,10,12,12,12,12
1990.0,64,64,64,64,64,64,64,64,64,58,64,64,64,64
1991.0,168,168,168,168,168,168,168,168,168,157,168,168,168,168
1992.0,485,485,485,485,485,485,485,485,485,382,484,485,485,485
1993.0,606,606,606,606,606,606,606,606,606,561,606,606,606,606
1994.0,642,642,642,642,642,642,642,642,642,488,640,642,642,642


### Gender
There are 3 distinguished gender categories:
- mare - female horse
- gelding - castrated male horse
- stallion - uncastrated male horse

The Polish names of the gender are translated into English and appended to the gender list:

In [ ]:
%%time
gender = []
for i in range(0,len(horses_DF)):
    if pd.isnull(horses_DF['gender_age'].iloc[i]):
        gender.append('')
        continue
    if 'klacz' in horses_DF['gender_age'].iloc[i]: 
        gender.append('mare')
    if 'wałach' in horses_DF['gender_age'].iloc[i]:
        gender.append('gelding')
    if 'ogier' in horses_DF['gender_age'].iloc[i]:
        gender.append('stallion')

One of the records is empty and need to be dealt with:

In [ ]:
pd.isnull(horses_DF['gender_age'].iloc[9295])

The length of the list is correct and can be added to data frame at column `gender`:

In [ ]:
len(gender)

In [ ]:
horses_DF['gender'] = gender

In [ ]:
horses_DF.head(2)

### Color
The most popular horse colors are:
- chestnut 
- gray
- bay
- black
- darkbay

And are assigned to each horse based on translation from Polish. Again Polish language is beautiful, so the name of the color differs for female and male horses.
If the horse has different color it would be 'not defined'.

In [ ]:
%%time
color = []
for i in range(0,len(horses_DF)):
    if pd.isnull(horses_DF['gender_age'].iloc[i]):
        print('?')
        color.append('')
        continue
    if ('Kasztanowaty' in horses_DF['gender_age'].iloc[i]) or ('Kasztanowata' in horses_DF['gender_age'].iloc[i]): 
        print('chestnut')
        color.append('chestnut')
        continue
    if ('Siwy' in horses_DF['gender_age'].iloc[i]) or ('Siwa' in horses_DF['gender_age'].iloc[i]): 
        print('gray')
        color.append('gray')
        continue
    if ('Gniady' in horses_DF['gender_age'].iloc[i]) or ('Gniada' in horses_DF['gender_age'].iloc[i]): 
        print('bay')
        color.append('bay')
        continue
    if ('Kary' in horses_DF['gender_age'].iloc[i]) or ('Kara' in horses_DF['gender_age'].iloc[i]): 
        print('black')
        color.append('black')
        continue
    if ('Ciemnogniady' in horses_DF['gender_age'].iloc[i]) or ('Ciemnogniada' in horses_DF['gender_age'].iloc[i]): 
        print('darkbay')
        color.append('darkbay')
        continue
    else:
        print(i,'inny')
        color.append('not_defined')

Let's see how many different color we have:

In [ ]:
list(set(color))

The length of the list is correct and can be added to data frame at column `color` and exported to csv file:

In [ ]:
len(color)

In [ ]:
horses_DF['color'] = color

The column `gender_age` is no longer needed, so we drop this column from data frame:

In [ ]:
horses_DF = horses_DF.drop('gender_age',axis=1)

In [ ]:
horses_DF.head(3)

In [ ]:
horses_DF.to_csv(r'.\horses_data\horses_DF.csv', encoding="utf-8")

### Country:
We use the package ***googletrans*** downloaded from the internet to translate the country of origin name into English. 

In [28]:
# The translation is needed in case of some of the data
# Example
from googletrans import Translator
translator = Translator()

tl = translator.translate('Stany Zjednoczone.')

In [29]:
tl.text

'United States.'

The most popular country of origin is Poland, so instead of translation it is faster to assign it straightaway. And the rest of the countries are translated by the ***googletrans*** package into English:

*There is a problem with this method and the internet connection. When `JSONDecodeError: Expecting value: line 1 column 1 (char 0)` reset the internet connection. 

In [ ]:
%%time
for i in range(0,len(horses_DF)):
    if horses_DF['country'].iloc[i]:
        if horses_DF['country'].iloc[i] == 'Polska':
            horses_DF['country'].iloc[i] = 'Poland'
        else:
            horses_DF['country'].iloc[i] = translator.translate(horses_DF['country'].iloc[i]).text

Export the data frame:

In [ ]:
horses_DF.to_csv(r'.\horses_data\horses_DF.csv', encoding="utf-8")

### Breed
Let's check all types of breeds that exist in scraped data:

In [ ]:
# All types of breed:
horses_DF.groupby('breed').sum()

Based on that information we have breeds:
- Pure blood Arabian
- Pure blood Anglo-Arabian
- Half blood Anglo-Arabian
- Pure blood English
- Half blood English
- Noble halfblood
- French trotter
- Polish breed - Malopolska
- Polish breed - Wielkopolska
- Polish breed - Slaska

In [ ]:
for i in range(0,len(horses_DF)):
    print(i)
    if pd.isnull(horses_DF['breed'].iloc[i]):
        continue
    if 'Czysta krew angloarabska' in horses_DF['breed'].iloc[i]:
        horses_DF['breed'].iloc[i] = 'Pure blood Anglo-Arabian'
        continue
    if 'arabska' in horses_DF['breed'].iloc[i]:
        horses_DF['breed'].iloc[i] = 'Pure blood Arabian'
        continue
    if 'francuski' in horses_DF['breed'].iloc[i]:
        horses_DF['breed'].iloc[i] = 'French trotter'
        continue
    if 'Małopolska' in horses_DF['breed'].iloc[i]:
        horses_DF['breed'].iloc[i] = 'Polish breed - Malopolska'
        continue
    if 'Pełna krew angielska' in horses_DF['breed'].iloc[i]:
        horses_DF['breed'].iloc[i] = 'Pure blood English'
        continue
    if 'Półkrew angielska' in horses_DF['breed'].iloc[i]:
        horses_DF['breed'].iloc[i] = 'Half blood English'
        continue
    if 'Półkrew angloarabska' in horses_DF['breed'].iloc[i]:
        horses_DF['breed'].iloc[i] = 'Half blood Anglo-Arabian'
        continue
    if 'Szlachetna' in horses_DF['breed'].iloc[i]:
        horses_DF['breed'].iloc[i] = 'Noble half blood'
        continue
    if 'Wielkopolska' in horses_DF['breed'].iloc[i]:
        horses_DF['breed'].iloc[i] = 'Polish breed - Wielkopolska'
        continue
    if 'śląska' in horses_DF['breed'].iloc[i]:
        horses_DF['breed'].iloc[i] = 'Polish breed - Slaska'
        continue
    else:
        continue

The horses_DF data frame is updated in the process.

### Trainer
Most of the horses in the data frame are not racing in the 2019 season, which is currently open. Those are not having any trainer for present season and are listed as 'Not in 2019'. For those racing in 2019 season the name of the trainer is provided. 

In [ ]:
for i in range(0,len(horses_DF)):
    print(i)
    if pd.isnull(horses_DF['trainer'].iloc[i]):
        continue
    if 'koń nie zgłoszony do sezonu 2019' in horses_DF['trainer'].iloc[i]:
        horses_DF['trainer'].iloc[i] = 'Not in 2019'
        continue
    else:
        continue

The final data frame is presented and exported to the csv file.

In [32]:
horses_DF.head(2)

,Unnamed: 0,id_name,breed,country,father,mother,trainer,owner,breeder,size,stables,name,dob,gender,color
0,0,20899,Pure blood English,France,"Johannesburg (USA, 1999)","A Dee Double You (USA, 2000) / Red Ransom",Not in 2019,Lokotrans,Haras de Bernesq,NaN,Lokotrans,A Dee Joe,2010.0,gelding,chestnut
1,1,12327,Pure blood English,United States,"Clever Champ (USA, 1982)","Mary Had a Lot (USA, 1985) / Double Zeus",Not in 2019,R. Rutkowski,A. H. Cunningham,NaN,Mutowo,A Lot Of Mary,1995.0,mare,darkbay


### Export the dataframe horses_DF to csv file:

In [ ]:
horses_DF.to_csv(r'.\horses_data\horses_DF.csv', encoding="utf-8")

### Import the dataframe horses_DF from csv file:

In [33]:
horses_DF = pd.read_csv(r'.\horses_data\horses_DF.csv',encoding="utf-8")

When exporting and importing the data frame the duplicate columns form, which can be deleted by:

In [ ]:
horses_DF = horses_DF.drop(['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1'],axis=1)

In [25]:
horses_DF.head(2)

,Unnamed: 0,id_name,breed,country,father,mother,trainer,owner,breeder,size,stables,name,dob,gender,color
0,0,20899,Pure blood English,France,"Johannesburg (USA, 1999)","A Dee Double You (USA, 2000) / Red Ransom",Not in 2019,Lokotrans,Haras de Bernesq,NaN,Lokotrans,A Dee Joe,2010.0,gelding,chestnut
1,1,12327,Pure blood English,United States,"Clever Champ (USA, 1982)","Mary Had a Lot (USA, 1985) / Double Zeus",Not in 2019,R. Rutkowski,A. H. Cunningham,NaN,Mutowo,A Lot Of Mary,1995.0,mare,darkbay


The final horses_DF data frame includes features:
- `id_name` : the horse id number 
- `breed` : the breed of the horse (10 breeds in total)
- `country` : the horse country of origin
- `father` : the horse father
- `mother` : the horse mother
- `trainer` : the name of the horse trainer when racing in 2019 season
- `owner` : the name of the person or stables that own the horse
- `breeder` : the name of the person or stables that breeded the horse
- `size` : the size of the horse if provided (height of the horse at the withers - chest circumference - circumference of the left leg above the furrow)
- `stables` : the horse stables
- `name` : the horse name
- `dob` : the horse date of birth (year)
- `gender` : the horse gender (3 genders)
- `color` : the horse color (5 colors in total)

---

## Results data frame
The results data frame will contain information about all the races through years 1995 and 2018, which covers 23 years of racing. This year (2019) the Suzewiec track celebrates the 80th anniversary of horse racing history, but only data from 1995 is available publicly.

The results are collected from 'https://koniewyscigowe.pl/wyniki_gonitw' and are divided accordingly to the city where the races took place. The numbers 1, 2, 3, 4 and 5 correspond to cities Warsaw, Wroclaw, Sopot, Stawiguda and Krakow. We are only interested for now in the results from Warsaw Sluzeiwec track. We need to loop through every season to scrap all the link with the race IDs. The links are the only thing that is interesting at this step, as they contain the information that will be used in further process.

In [34]:
%%time
it = 0
race = []
size = len(range(1995,2019))-1
for year in range(1995,2019):
    http = httplib2.Http()
    # miasto = 1 - Warszawa
    # miasto = 2 - Wroclaw
    # miasto = 3 - Sopot
    # miasto = 4 - Stawiguda
    # miasto = 5 - Krakow
    status, response = http.request('https://koniewyscigowe.pl/wyniki_gonitw?miasto=1&sezon='+str(year))
    for link in BeautifulSoup(response, 'html.parser', parse_only=SoupStrainer('a')):
        if link.has_attr('href'):
            if (re.findall(r'\b'+'wyscig'+r'\b',link['href'])):
                race.append(link['href'])
                #print(year, len(race))
                          
    # Progress bar
    string = '['
    for k in range(int(it / size * 100)):
        string += '#'
    for k in range(100 - int(it / size * 100)):
        string += '-'
    string += ']'
    string += '{0:.2f}'.format(it / size * 100) + "%"
    print('\r' + string, sep='', end=' ', flush=True)
    it += 1
    
print('\n')

[####################################################################################################]100.00% 

Wall time: 19.6 s


We can check how many results we have:

In [35]:
len(race)

26784

Unfortunately each link is doubled, because the number of the race and the date of the race on the webpage both linked to the same webpage for the details of that race. Set will deal with it, as it drop the duplicates: 

In [36]:
race[0:7]

['/wyscig?w=13206-tor-służewiec',
 '/wyscig?w=13206-tor-służewiec',
 '/wyscig?w=13207-tor-służewiec',
 '/wyscig?w=13207-tor-służewiec',
 '/wyscig?w=13208-tor-służewiec',
 '/wyscig?w=13208-tor-służewiec',
 '/wyscig?w=13209-tor-służewiec']

In [37]:
race_set = set(race)
len(race_set)

13392

And we are left with the proper length of the results data frame, where the records do not double:

In [38]:
race = list(race_set)
race[0:3]

['/wyscig?w=2258-tor-służewiec',
 '/wyscig?w=756-tor-służewiec',
 '/wyscig?w=15428-tor-służewiec']

The next step will be to explore one of the link from the race list. We choose randomly the id 3620:

In [39]:
r = pd.read_html('https://koniewyscigowe.pl/wyscig?w='+'3620')

The first table contains lot of information:

In [40]:
r[0]

,0
0,"Pogodnie, temp. +22°C, tor lekko elastyczny (3.2)"
1,Gonitwa dla 3 l. koni czystej krwi arabskiej I...


In [41]:
# The information about the distance, total price for the race, and the race category:
r[0].iloc[1][0]

'Gonitwa dla 3 l. koni czystej krwi arabskiej II grupy. Dystans 1600m •\xa0Pula nagród: 5\xa0950 zł •\xa0Grupa II'

In [42]:
# The information about the temperature and the track elasticity:
r[0].iloc[0][0]

'Pogodnie, temp. +22°C, tor lekko elastyczny (3.2)'

The second table shows the detailed information about final standing, the lengths at the finishing line, the horse, its age, the jockey, the jockeys weights, trainers and individual prices:

In [46]:
r[1]

,Unnamed: 0,odl.,koń,wiek,jeździec,waga,trener,wygrana
0,1,NaN,Ehud,3,dż. A. Turgaev,58,M. Łojek,3 400 zł
1,2,2½,Bakalia,3,dż. M. Pilich,56,A. Wyrzyk,1 360 zł
2,3,5,Edinet,3,dż. A. Reznikov,58,A. Nieora-Tchkuaseli,680 zł
3,4,3½,Efata,3,dż. W. Szymczuk,56,T. Kluczyński,340 zł
4,5,16,Makat,3,dż. V. Popov,58,B. Ziemiański,170 zł
5,6,6,Saber,3,dż. S. Wasiutow,58,D. Kałuba,0 zł
6,7,12,Bajan,3,k. dż. K. Gembicka,58,D. Kałuba,0 zł
7,8,19,Kahil,3,dż. J. Ochocki,58,T. Kluczyński,0 zł


In [44]:
# The information about the number of horses in the race:
len(r[1])

8

In [45]:
# The information about the winner:
r[1].iloc[0][2]

'Ehud'

The third and final table for this webpage contains more detailed information about the time of the race, style of the race and bets:

In [47]:
r[2]

,0
0,Czas : 1'54.70 • (7.8-36-36.4-34.5) • Styl:wys...
1,ZWC: 3.70 zł • PDK: 51.10 zł • TRJ: 434.20 zł ...


In [48]:
# The information about the time records for the race and race style:
r[2].iloc[0][0]

"Czas : 1'54.70\xa0•\xa0(7.8-36-36.4-34.5)\xa0• Styl:wysyłany"

In [49]:
# The information about the bets:
r[2].iloc[1][0]

'ZWC: 3.70 zł • PDK: 51.10 zł • TRJ: 434.20 zł • CZW: 572.50 zł • TPL: zł • KWN: zł'

For the chosen race we can display the information that is interesting and will be added to the results data frame we are building:

In [50]:
import re
s = r[0].iloc[1][0]
result_dist = re.search('Dystans (.*)m', s)
result_price = re.search('Pula nagród: (.*) zł ', s)
result_cat = re.search('Grupa (.*)', s)
print('Distance: ',result_dist.group(1))
print('Price: ',result_price.group(1).replace('\xa0',''))
print('Category: ',result_cat.group(1))
w = r[0].iloc[0][0]
result_temp = re.search('\+(.*)°C', w)
result_track = re.search('\((.*)\)', w)
print('Temperature: ',result_temp.group(1))
print('Track elasticity: ',result_track.group(1))
result_numb = len(r[1])
print('Number of horses: ',result_numb)
result_win = r[1].iloc[0][2]
print('Winner: ', result_win)
t = r[2].iloc[0][0]
result_time = re.search('Czas : (.*)0\xa0•', t)
print('Time: ',result_time.group(1))
b = r[2].iloc[1][0]
result_bet = re.search('ZWC: (.*) zł • PDK',b)
print('Bet: ',result_bet.group(1))

Distance:  1600
Price:  5950
Category:  II
Temperature:  22
Track elasticity:  3.2
Number of horses:  8
Winner:  Ehud
Time:  1'54.7
Bet:  3.70


This way we created 9 features: `distance`, `price`, `category`, `temperature`, `track elasticity`, ` number of horses`, `winner`, `time` and `bet`.

Date and day of the season:

The information about the date and which day of the season it is, is included in the page title. Previously we were scraping tables and link. Scraping title is different. 
For the same race id 3620 we look for header of type h3 where the information we need is. And from there we get the day of the season and the date of the race. 

In [51]:
http = httplib2.Http()
status, response = http.request('https://koniewyscigowe.pl/wyscig?w='+'3620')
bs = BeautifulSoup(response, 'html.parser', parse_only=SoupStrainer('h3'))

In [52]:
bs = str(bs)
bs = bs.split('</h3><h3')
print(bs[0])

<h3 class="g-color-black g-font-weight-600 mb-3">Wynik gonitwy (165) Warszawa - tor Służewiec, Dzień 20 - Niedziela 31 Sierpienia 2008


In [53]:
result_day_date = re.search('Dzień (.*)', bs[0])
result_day_date = result_day_date.group(1).split(' - ')
result_day = result_day_date[0]
result_date = result_day_date[1]
print('Day of season: ', result_day)
print('Date: ', result_date)

Day of season:  20
Date:  Niedziela 31 Sierpienia 2008


And that is it. The day of the season is 20, and the date in in Polish, which will be translated later, when cleaning the data frame.

### Bonus: 
How to find the horse by its name?

In [31]:
# Find a horses by its name
for i in range(0,len(horses_DF)):
    if horses_DF.iloc[i]['name'] == 'Evergonnatouchthis':
         print(i,horses_DF.iloc[i])

6383 Unnamed: 0                      6383
Unnamed: 0.1                    6383
id_name                        20375
breed             Pure blood Arabian
country                       Polska
father          NivourGonnaTouchThis
mother                Elsana / Gafal
trainer                  Not in 2019
owner                    J. Głowacki
breeder                  J. Głowacki
size                             NaN
stables                    Mała Wieś
name              Evergonnatouchthis
dob                             2012
gender                      stallion
color                           gray
Name: 6383, dtype: object


### Create the Results Data Frame
With the available information we build the second data frame for races results.

In [54]:
results_df = pd.DataFrame(columns = ['id_race', 'date', 'day', 'distance', 'category',
                                   'winner', 'number_of_horses', 'price',
                                    'temperature', 'track', 'time', 'bet']) 
results_df

,id_race,date,day,distance,category,winner,number_of_horses,price,temperature,track,time,bet


### Id
From the previously created race list we want to get just the number. Form the we split the string of the record and save only the number in the previous list.

In [55]:
# Extract id_race
race = list(race_set)
s = race[1]
s

'/wyscig?w=756-tor-służewiec'

In [56]:
race[0:3]

['/wyscig?w=2258-tor-służewiec',
 '/wyscig?w=756-tor-służewiec',
 '/wyscig?w=15428-tor-służewiec']

In [57]:
%%time
for i in range(0,len(race)):
    temp = race[i].split('=')
    temp = temp[1].split('-')
    race[i] = temp[0]

Wall time: 12 ms


In [58]:
race[0:3]

['2258', '756', '15428']

In [59]:
len(race)

13392

The number can be then added to the data frame column `id_race`:

In [60]:
for i in range(0,len(race)):
    results_df.at[i,'id_race'] = race[i]

In [61]:
results_df.head()

,id_race,date,day,distance,category,winner,number_of_horses,price,temperature,track,time,bet
0,2258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,756,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15428,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8415,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Distance, price, category, temperature, track elasticity, number of horses, winner, time, bet, date and day of the season, all added to the Results Data Frame in one loop

In [ ]:
%%time
import re
http = httplib2.Http()

it = 0
size = len(race)

for i in race:
    print(i)
    r = pd.read_html('https://koniewyscigowe.pl/wyscig?w='+str(i))
    
    s = r[0].iloc[1][0]
    #print(s)
    result_dist = re.search('Dystans (.*)m', s)
    result_price = re.search('Pula nagród: (.*) zł ', s)
    result_cat = re.search('Grupa (.*)', s)
    #print('Distance: ',result_dist.group(1))
    #print('Price: ',result_price.group(1).replace('\xa0',''))
    #print('Category: ',result_cat.group(1))
    w = r[0].iloc[0][0]
    #print(w)
    result_temp = re.search('\+(.*)°C', w)
    result_track = re.search('\((.*)\)', w)
    #print('Temperature: ',result_temp.group(1))
    #print('Track elasticity: ',result_track.group(1))
    result_numb = len(r[1])
    #print('Number of horses: ',result_numb)
    result_win = r[1].iloc[0][2]
    #print('Winner: ', result_win)
    t = r[2].iloc[0][0]
    result_time = re.search('Czas : (.*)0\xa0•', t)
    #print('Time: ',result_time.group(1))
    b = r[2].iloc[1][0]
    result_bet = re.search('ZWC: (.*) zł • PDK',b)
    #print('Bet: ',result_bet.group(1))
    
    results_df.at[i,'id_race'] = i
    if result_dist:
        results_df.at[i,'distance'] = result_dist.group(1)
    if result_price:
        results_df.at[i,'price'] = result_price.group(1).replace('\xa0','')
    if result_cat:
        results_df.at[i,'category'] = result_cat.group(1)
    if result_temp:
        results_df.at[i,'temperature'] = result_temp.group(1)
    if result_track:
        results_df.at[i,'track'] = result_track.group(1)
    if result_numb:
        results_df.at[i,'number_of_horses'] = result_numb
    if result_win:
        results_df.at[i,'winner'] = result_win
    if result_time:
        results_df.at[i,'time'] = result_time.group(1)
    if result_bet:
        results_df.at[i,'bet'] = result_bet.group(1)
    
    status, response = http.request('https://koniewyscigowe.pl/wyscig?w='+str(i))
    bs = BeautifulSoup(response, 'html.parser', parse_only=SoupStrainer('h3'))
    
    bs = str(bs)
    bs = bs.split('</h3><h3')
    #print(bs[0])
    
    result_day_date = re.search('Dzień (.*)', bs[0])
    result_day_date = result_day_date.group(1).split(' - ')
    result_day = result_day_date[0]
    result_date = result_day_date[1]
    #print('Day of season: ', result_day)
    #print('Date: ', result_date)
    
    if result_day:
        results_df.at[i,'day'] = result_day
    if result_date:
        results_df.at[i,'date'] = result_date
    
    # Progress bar
    string = '['
    for k in range(int(it / size * 100)):
        string += '#'
    for k in range(100 - int(it / size * 100)):
        string += '-'
    string += ']'
    string += '{0:.2f}'.format(it / size * 100) + "%"
    print('\r' + string, sep='', end=' ', flush=True)
    it += 1
    

### Cleaning the Results Data Frame and some more information

### Date
To change the date format from : 'Niedziela 27 Września 2009' to : '2009-09-27 00:00:00' for the first record and all the other dates we need to perform an operation. 

The first word in string is a name of the day of the week. Nowadays the races are only on the weekends but apparently back in 1995 and 1996 they had races on Wednesdays. So the first part of the loop translates the name of the day of the week. If different day is submitted the default value of 'Sunday' will be assigned.

The second part of the string is the number of the day. 

The third part of the string is the name of the month. The horse racing season is always between the months of April and November. In Polish there exist different alterations of the names so they are included. 

The forth part of the string is the year of the race. 

After the translation into English, the module ***datetime*** is used to convert into date format.

In [ ]:
from datetime import datetime

for i in range(0,len(results_df)):
    datetime_str = results_df['date'].iloc[i]
    datetime_spl = datetime_str.split(' ')
    # Days: (usually on the weekends)
    if datetime_spl[0] == 'Niedziela':
        datetime_spl[0] = 'Sunday'
    elif datetime_spl[0] == 'Sobota':
        datetime_spl[0] = 'Saturday'
    elif datetime_spl[0] == 'Środa':
        datetime_spl[0] = 'Wednesday'  # apparently back in 1995 and 1996 they had races on Wednesdays
    else:
        datetime_spl[0] = 'Sunday' # default
    # Months: (the races are always between April and November)
    # The Polish language is beautiful and requires different forms of the words
    if datetime_spl[2] == 'Listopada' or datetime_spl[2] == 'Listopad':
        datetime_spl[2] = 'November'
    elif datetime_spl[2] == 'Października' or datetime_spl[2] == 'Październik':
        datetime_spl[2] = 'October'
    elif datetime_spl[2] == 'Września' or datetime_spl[2] == 'Wrzesień':
        datetime_spl[2] = 'September'
    elif datetime_spl[2] == 'Sierpnia' or datetime_spl[2] == 'Sierpień' or datetime_spl[2] == 'Sierpienia':
        datetime_spl[2] = 'August'
    elif datetime_spl[2] == 'Lipca' or datetime_spl[2] == 'Lipiec':
        datetime_spl[2] = 'July'
    elif datetime_spl[2] == 'Czerwca'  or datetime_spl[2] == 'Czerwiec':
        datetime_spl[2] = 'June'
    elif datetime_spl[2] == 'Maja' or datetime_spl[2] == 'Maj':
        datetime_spl[2] = 'May'
    elif datetime_spl[2] == 'Kwietnia' or datetime_spl[2] == 'Kwiecień':
        datetime_spl[2] = 'April'
    else:
        datetime_spl[2] = 'August' # default
    datetime_str = ' '.join(datetime_spl)
   
    results_df['date'].iloc[i] = datetime.strptime(datetime_str, '%A %d %B %Y')

In [68]:
results_df['date'].iloc[0]

'2009-09-27 00:00:00'

Let's check the info for the data frame:

In [69]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13392 entries, 0 to 13391
Data columns (total 15 columns):
Unnamed: 0          13392 non-null int64
Unnamed: 0.1        13392 non-null int64
Unnamed: 0.1.1      13392 non-null int64
id_race             13392 non-null int64
date                13392 non-null object
day                 13392 non-null int64
distance            13392 non-null int64
category            11382 non-null object
winner              13392 non-null object
number_of_horses    13392 non-null int64
price               13391 non-null float64
temperature         13392 non-null object
track               13392 non-null object
time                13392 non-null object
bet                 12008 non-null float64
dtypes: float64(2), int64(7), object(6)
memory usage: 1.5+ MB


Because `temperature`, `track` and `time` are object rather than numeric values we need to convert them.

### Time 
We want to convert the time from string format for example for record 88: "2'10.0" into seconds in this case 130. For that we multiply the minutes with 60 and add the remaining seconds.

In [74]:
results_df['time'].iloc[88]

"2'10.0"

In [75]:
time_spl = results_df['time'].iloc[88].split('\'')
time_spl

['2', '10.0']

In [76]:
time_sec = int(time_spl[0])*60 + float(time_spl[1]) 
time_sec

130.0

### Track

The track score from 0 to 5 for the ground elasticity:
- hard (0.1-1.5) 
- light (1.6-2.5) 
- slightly elastic (2.6-3.2) 
- flexible (3.3-4) 
- very flexible (4.1-5) 
- heavy (over 5)

The track condition is determined before the start of each race day and announced to the public. This does not exclude subsequent modifications when the atmospheric conditions change. There are six basic types of track, which are given in order from the most dry one. Horses run the fastest on the hard dry track, and the slowest on the heavily elastic track. In the case of prolonged drought the track may be hard. Such a condition is very bad for animal health and can cause contusion. The organizers try to avoid this situation and water the track. Immediately after torrential rain the light or slightly elastic track may turn into slippery (no longer dry yet not elastic). In late autumn, when there are frosts, the track may be frozen, and after snowfall - snow-covered. The frozen track is similar to hard, but due to its slipperiness, horses run slower. The snowy track resembles a very flexible, heavy surface.

We simply transform the string value into float. In some cases the data is saved with additional information, which will not work with float method. For that reason all the data longer than 3 characters are deleted for now.

### Temperature
We simply transform the string value into integer. In some cases the data is saved with comma rather then period, which will not work with int method. For that reason we check if the comma exist in the temperature string before the conversion.

The transformation loop for the `track`, `temperature` and `time`:

In [ ]:
%%time
it = 0
size = len(results_df)-1
for i in range(0,len(results_df)):
    # track
    if results_df['track'].iloc[i] and len(results_df['track'].iloc[i])<4:
        results_df['track'].iloc[i] = float(results_df['track'].iloc[i])
    else: 
        results_df['track'].iloc[i] = ' '
    # temperature
    if pd.isnull(results_df['temperature'].iloc[i]) == 0 and ',' not in results_df['temperature'].iloc[i]:
        results_df['temperature'].iloc[i] = int(results_df['temperature'].iloc[i])
    else: 
        results_df['temperature'].iloc[i] = ' '
    # time
    if pd.isnull(results_df['time'].iloc[i]) == 0:
        time_spl = results_df['time'].iloc[i].split('\'')
        results_df['time'].iloc[i] = int(time_spl[0])*60 + float(time_spl[1]) 
    else:
        results_df['time'].iloc[i] = ' '
        
    # Progress bar
    string = '['
    for k in range(int(it / size * 100)):
        string += '#'
    for k in range(100 - int(it / size * 100)):
        string += '-'
    string += ']'
    string += '{0:.2f}'.format(it / size * 100) + "%"
    print('\r' + string, sep='', end=' ', flush=True)
    it += 1

There are some NaN values in the data frame, we do not want them, as they may cause errors when encountered. 

In [83]:
results_DF.fillna('')

,id_race,date,day,distance,category,winner,number_of_horses,price,temperature,track,time,bet
0,3144,2009-09-27 00:00:00,44,1600,IV,Salinka,7,4900,22,2.8,99.4,6.6
1,5084,2005-06-12 00:00:00,9,1600,II,Ghazallah,7,3570,16,2.8,111.4,1.8
2,213,2014-07-06 00:00:00,21,3800,,Persei,6,17500,28,1.8,253.1,13.1
3,4391,2007-11-03 00:00:00,48,1600,III,Defa,6,5600,6,3.5,102.5,7.12
4,11883,1997-09-21 00:00:00,58,1600,IV,Aksamit,7,2625,16,2.7,102.4,5.5
5,7126,2003-08-02 00:00:00,30,1400,,Dandolo,10,54250,27,3.5,86.7,7.8
6,8446,2001-09-09 00:00:00,49,1600,IV,Mystery,6,4312,,3.5,104.2,11.65
7,4716,2006-10-08 00:00:00,16,1600,IV,Merlini,7,3060,14,3.5,101.8,
8,9127,2000-07-26 00:00:00,31,1200,IV,Doria Andrea,7,3850,24,3.5,75.2,8.6
9,10214,1999-10-09 00:00:00,70,1800,III,Glorietta,7,3850,11,3.5,131.3,30.1


The final data frame is presented and exported to the csv file.

In [82]:
results_df.head(7)

,id_race,date,day,distance,category,winner,number_of_horses,price,temperature,track,time,bet
0,3144,2009-09-27 00:00:00,44,1600,IV,Salinka,7,4900.0,22,2.8,99.4,6.60
1,5084,2005-06-12 00:00:00,9,1600,II,Ghazallah,7,3570.0,16,2.8,111.4,1.80
2,213,2014-07-06 00:00:00,21,3800,NaN,Persei,6,17500.0,28,1.8,253.1,13.10
3,4391,2007-11-03 00:00:00,48,1600,III,Defa,6,5600.0,6,3.5,102.5,7.12
4,11883,1997-09-21 00:00:00,58,1600,IV,Aksamit,7,2625.0,16,2.7,102.4,5.50
5,7126,2003-08-02 00:00:00,30,1400,NaN,Dandolo,10,54250.0,27,3.5,86.7,7.80
6,8446,2001-09-09 00:00:00,49,1600,IV,Mystery,6,4312.0,,3.5,104.2,11.65


### Export the dataframe results_DF to csv file:

In [ ]:
results_df.to_csv(r'.\horses_data\results_DF.csv', encoding="utf-8")

### Import the dataframe results_DF from csv file:

In [79]:
results_DF = pd.read_csv(r'.\horses_data\results_DF.csv',encoding="utf-8")

When exporting and importing the data frame the duplicate columns form, which can be deleted by:

In [81]:
results_DF = results_DF.drop(['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1'],axis=1)

The final results_DF data frame includes features:
- `id_race`: the dedicated number to each race during the race day
- `date`: the day of the race - day/month/year
- `day`: the number of the race of the season (total number of races per season differs from season to season)
- `distance`: the distance of the race in meters
- `category`: the category of the race based on horses categories at that time - in order from the best: A, B, I, II, III, IV 
- `winner`: the winning horse
- `number_of_horses`: the overall number of horses in the race
- `price`: the price for the winning horse, do not mistake with the bet on the horse
- `temperature`: the temperature during the race in Celsius degrees
- `track`: the track score from 0 to 5 for the ground elasticity - hard (0.1-1.5) - light (1.6-2.5) - slightly elastic (2.6-3.2) - flexible (3.3-4) - very flexible (4.1-5) - heavy (over 5). 
- `time`: the total race duration time for the winning horse
- `bet`: the price of the winning bet

---

## Races data frame

The Races Data Frame will contain information about all the races from the Results Data Frame with detailed information about the standings in each of them.

The results are collected from 'https://koniewyscigowe.pl/wyscig'.

### Create the Races Data Frame
With the available information we build the third data frame for races.

In [86]:
# Races dataframe:
races_df = pd.DataFrame(columns = ['id_race', 'place', 'lengths', 'horse', 'age', 'jockey', 'weight', 'trainer', 'award'] )
                      
races_df 

,id_race,place,lengths,horse,age,jockey,weight,trainer,award


The next step will be to explore one of the link. We choose randomly the id 2576 from the race list collected in Results Data Frame:

In [91]:
r = pd.read_html('https://koniewyscigowe.pl/wyscig?w='+'2576')

That is once again the link we explored in Results Data Frame, but this time we want to look at the second table with all the results for the race:

In [92]:
r[1]

,Unnamed: 0,odl.,koń,wiek,jeździec,waga,trener,wygrana
0,1,NaN,San-Miego,11,dż. S. Wasiutow,62,M. Malinowski,2 100 zł
1,2,1¼,Pola,4,dż. E. Zahariev,58,E. Zahariev,1 155 zł
2,3,2,Laazim,5,dż. V. Popov,62,K. Zawiliński,630 zł
3,4,1½,Falster,4,dż. P. Piątkowski,59,M. Janikowski,315 zł
4,5,¾,Frou-Frou,5,dż. A. Turgaev,62,M. Łojek,105 zł
5,6,5,Betola,4,k. dż. L. Reznikova-Wróblewska,56,M. Janikowski,0 zł
6,7,łeb,Ma Albigowa,4,dż. A. Reznikov,57,A. Nieora-Tchkuaseli,0 zł
7,8,2,Katar,4,pr. dż. M. Březina,57,M. Janikowski,0 zł
8,9,19,Eh Hago,4,dż. R. G.-Panczew,59,M. Malinowski,0 zł


We have 9 horses in total and 8 columns. The columns have names in Polish. 

In [93]:
len(r[1])

9

In [94]:
len(r[1].columns)

8

The horse on the second position is 'Pola':

In [95]:
r[1].iloc[1][2]

'Pola'

### Filling the Races Data Frame
We prepare the script to scrap the date from that table and save it to our data frame. The process is pretty easy, we only want to make change in the award column as the symbol '\xa0' needs to be deleted.

In some of the races (to be exact in 166 races) there is a table in different format for some reason. Maybe there was a person who had his/hers own vision, and created a different table. The data in both examples of the table is almost the same, the second version lack information about the lengths on the finishing line and about the weights of the load. The versions differ in the column number and for that reason in the script this fact is acknowledged.

The loop is again timed to show how long it takes to process it and the progress bar is displayed for the user.

In [ ]:
%%time
import re

l = 0
wr = 0

it = 0
size = len(race)

for i in race:
    print(i)
    r = pd.read_html('https://koniewyscigowe.pl/wyscig?w='+str(i))
    s = r[1]
    print(len(r[1].columns))
    
    if len(r[1].columns) == 8:
    
        for j in range(0,len(r[1])):
            race_place = r[1].iloc[j][0]
            race_lens = r[1].iloc[j][1]
            race_horse = r[1].iloc[j][2]
            race_age = r[1].iloc[j][3]
            race_joc = r[1].iloc[j][4]
            race_weight = r[1].iloc[j][5]
            race_train = r[1].iloc[j][6]
            race_award = r[1].iloc[j][7].replace('\xa0','')

            races_df.at[l,'id_race'] = i
            if race_place:
                races_df.at[l,'place'] = race_place
            if race_lens:
                races_df.at[l,'lengths'] = race_lens
            if race_horse:
                races_df.at[l,'horse'] = race_horse
            if race_age:
                races_df.at[l,'age'] = race_age
            if race_joc:
                races_df.at[l,'jockey'] = race_joc
            if race_weight:
                races_df.at[l,'weight'] = race_weight
            if race_train:
                races_df.at[l,'trainer'] = race_train
            if race_award:
                races_df.at[l,'award'] = race_award

            l = l + 1
            #print(l)

    
    elif len(r[1].columns) == 7:
    
        for j in range(1,len(r[1])):
            race_place = r[1].iloc[j][0]
            race_horse = r[1].iloc[j][2]
            race_age = r[1].iloc[j][3]
            race_joc = r[1].iloc[j][4]
            race_train = r[1].iloc[j][5]
            race_award = r[1].iloc[j][6].replace('\xa0','')

            races_df.at[l,'id_race'] = i
            if race_place:
                races_df.at[l,'place'] = race_place
            if race_horse:
                races_df.at[l,'horse'] = race_horse
            if race_age:
                races_df.at[l,'age'] = race_age
            if race_joc:
                races_df.at[l,'jockey'] = race_joc
            if race_train:
                races_df.at[l,'trainer'] = race_train
            if race_award:
                races_df.at[l,'award'] = race_award

            l = l + 1
            #print(l)
            
    else:
        wr += 1
        
    # Progress bar
    string = '['
    for k in range(int(it / size * 100)):
        string += '#'
    for k in range(100 - int(it / size * 100)):
        string += '-'
    string += ']'
    string += '{0:.2f}'.format(it / size * 100) + "%"
    print('\r' + string, sep='', end=' ', flush=True)
    it += 1
        
print('Wrong format tables number: ',wr)

We display the data frame and its length:

In [99]:
races_df.head()

,Unnamed: 0,id_race,place,lengths,horse,age,jockey,weight,trainer,award
0,0,14962,1,NaN,Ss Mothill,5,dż. S. Vasyutov,62,B. Strójwąs,4800zł
1,1,14962,2,2½,Pirat,8,dż. M. Srnec,62,A. Walicki,1920zł
2,2,14962,3,2½,Hasty,4,dż. P. Krowicki,57,M. Kosicki,960zł
3,3,14962,4,2½,Sandor,5,dż. A. Turgaev,62,J. Głowacki,480zł
4,4,14962,5,szyja,Tollee Dust,10,dż. C. Fraisl,62,M. Romanowski,240zł


In [100]:
len(races_df)

99748

### Cleaning the Races Data Frame
The lengths column definitely needs some changes as the quarter and half symbols and polish names do not give as much information.

### Lengths
A horse length, or simply length, is a unit of measurement for the length of a horse from nose to tail, approximately 8 feet (2.4 m). 

Abbreviations:
- Nose - nse - 0.05L
- Short head - sh - 0.1L
- Head - hd - 0.2L
- Short neck - snk - 0.25L
- Neck - nk - 0.3L
- Half a length - 0.5L
- Three-quarters of a length - 0.75L
- One length - 1L
- Distance - dst

It would be easier to convert the column `lengths` to all numeric values.

To see how many different lengths are in the Races Data Frame we count them:

In [101]:
races_df.groupby('lengths').count()

,Unnamed: 0,id_race,place,horse,age,jockey,weight,trainer,award
lengths,,,,,,,,,
---,782,782,779,782,782,782,782,782,782
1,5889,5889,5889,5889,5889,5889,5889,5889,5889
1.0,203,203,203,203,203,203,203,203,203
10,811,811,811,811,811,811,811,811,811
10.0,33,33,33,33,33,33,33,33,33
10¼,11,11,11,11,11,11,11,11,11
10½,17,17,17,17,17,17,17,17,17
10¾,8,8,8,8,8,8,8,8,8
11,594,594,594,594,594,594,594,594,594


Let's explore one of the lengths data:

In [102]:
races_df['lengths'].iloc[1][0]

'2'

In [103]:
races_df['lengths'].iloc[1][1]

'½'

In [104]:
races_df['lengths'].iloc[1]

'2½'

Instead of the half symbol we simply put 0.5 and change the string into a numeric value:

In [106]:
int(races_df['lengths'].iloc[1][:-1])+0.5

2.5

The symbols and abbreviations are changed into numeric values in the script:

In [ ]:
%%time
it = 0
size = len(races_df)

for i in range (0,len(races_df)):
    #print(i)
    # nan - first horse
    if pd.isnull(races_df['lengths'].iloc[i]):
        races_df['lengths'].iloc[i] = 0
    # max dist
    elif races_df['lengths'].iloc[i] == 'daleko':
        races_df['lengths'].iloc[i] = 99
    # neck
    elif races_df['lengths'].iloc[i] == 'szyja':
        races_df['lengths'].iloc[i] = 0.3
    # short neck
    elif races_df['lengths'].iloc[i] == 'kr. szyja':
        races_df['lengths'].iloc[i] = 0.25
    # head
    elif races_df['lengths'].iloc[i] == 'łeb':
        races_df['lengths'].iloc[i] = 0.2
    # short head
    elif races_df['lengths'].iloc[i] =='kr. łeb':
        races_df['lengths'].iloc[i] = 0.1
    # nose
    elif races_df['lengths'].iloc[i] == 'nos':
        races_df['lengths'].iloc[i] = 0.05
    # halfs
    elif '½' in races_df['lengths'].iloc[i]:
        if races_df['lengths'].iloc[i][:-1]:
            races_df['lengths'].iloc[i] = int(races_df['lengths'].iloc[i][:-1])+0.5
        else: 
            races_df['lengths'].iloc[i] = 0.5
    # quarters
    elif '¾' in races_df['lengths'].iloc[i]:
        if races_df['lengths'].iloc[i][:-1]:
            races_df['lengths'].iloc[i] = int(races_df['lengths'].iloc[i][:-1])+0.75
        else:
            races_df['lengths'].iloc[i] = 0.75
        
    
    # Progress bar
    string = '['
    for k in range(int(it / size * 100)):
        string += '#'
    for k in range(100 - int(it / size * 100)):
        string += '-'
    string += ']'
    string += '{0:.2f}'.format(it / size * 100) + "%"
    print('\r' + string, sep='', end=' ', flush=True)
    it += 1

The final data frame is presented and exported to the csv file.

In [107]:
races_df.head(5)

,Unnamed: 0,id_race,place,lengths,horse,age,jockey,weight,trainer,award
0,0,14962,1,0,Ss Mothill,5,dż. S. Vasyutov,62,B. Strójwąs,4800zł
1,1,14962,2,2.5,Pirat,8,dż. M. Srnec,62,A. Walicki,1920zł
2,2,14962,3,2.5,Hasty,4,dż. P. Krowicki,57,M. Kosicki,960zł
3,3,14962,4,2.5,Sandor,5,dż. A. Turgaev,62,J. Głowacki,480zł
4,4,14962,5,0.3,Tollee Dust,10,dż. C. Fraisl,62,M. Romanowski,240zł


### Export the dataframe races_DF to csv file:

In [108]:
races_df.to_csv(r'.\horses_data\races_DF.csv', encoding="utf-8")

### Import the dataframe races_DF from csv file:

In [109]:
races_df = pd.read_csv(r'.\horses_data\races_DF.csv',encoding="utf-8")

The final races_DF data frame includes features:
- `id_race` : the dedicated number to each race during the race day
- `place` : the number of standing in the race
- `lengths` : the lengths between horses on the finishing line
- `horse` : the name of the horse
- `age` : the age of the horse
- `jockey` : the jockey riding the horse in the race
- `weights` : the weight of the total load (jockey + added weights)
- `trainer` : the trainer of the horse in the season
- `award` : the award for the horse in the race on the final position, based on the total price for that race